# Node Classification Version GCN

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import cv2
import argparse
import torch
import tqdm
import random
from torchsummary import summary
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from glob import glob
import seaborn as sns
from torchsummary import summary

from models.DAG import DeepAdaptiveGraph
from Dataset import *
from torchvision import transforms
from preprocessing import *
from trainer import experiment
from settings import Abs_L2_Adj_backbone_hrnet32_Aug_512x512_Tanh_DoubleGCN

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False 

args = Abs_L2_Adj_backbone_hrnet32_Aug_512x512_Tanh_DoubleGCN.args

transform_train = transforms.Compose([
    Gamma_2D(),
    Shift_2D(),
#     RandomBrightness(),
    Rotation_2D(),
    RandomSharp(),
    RandomBlur(),
    RandomNoise(),
    ToTensor(),
    ])

transform_test = transforms.Compose([
    ToTensor(),
])

trainset = Dataset_Image_Point_Test(args.data.train, 
                               mode = "train", 
                               transform=transform_train, 
                               train_size=(args.img_height, args.img_width))

validset = Dataset_Image_Point_Test(args.data.valid, 
                               mode = "valid", 
                               transform=transform_test, 
                               train_size=(args.img_height, args.img_width))

train_loader = DataLoader(trainset, batch_size=args.batch_size, shuffle=args.shuffle)
valid_loader = DataLoader(validset, batch_size=1, shuffle=False)

In [ ]:
model = DeepAdaptiveGraph(args, absolute = True, L2 = True, softmax = False, sigmoid = False, clip = False)
# weight = torch.load('./result/Abs_L2_Adj_backbone_hrnet32_Aug_512x512_Tanh/weights/best155_model.pth')
# model.load_state_dict(weight)

logs = experiment(model, train_loader, valid_loader, args)